In [44]:
interm_dir3 = tmp_path+ds_type+'/neg/'
interm_dir2 = tmp_path+ds_type+'/pos/'
fname_df = 'df_'+ds_type+'.csv'

random.seed(313)
for scan_id in scan_id_:#[xx+1:]:
    scan_1 = ctscan(scan_id) 
    S,B=get_segmented_lungs2(scan_1.image_resampled, plot=False)

    T = B.shape[1]**2
    Areas=[np.sum(b)/T for b in B]
    ind2=[i for i,a in enumerate(Areas) if a>.02]
    z1,z2=ind2[0],ind2[-1]

    for k in range(10):
        zz=np.random.randint(z1,z2)

        Bf=B[zz].flatten()
        #In that slice, find the elements that are true
        Cs=[i for i,e in enumerate(Bf) if e]
        #randomly select an element from Cs
        i = random.choice(Cs)
        #from i get the original row and column of that element in B
        a=B.shape[1];a
        r = i//a
        c=i-a*r

        #Thus, we have succssfully selected a random point that resides inside the lung area
        #we would like to extract a 52x52x52 patch from the ctscan volume.
        #The patch is centered at the conditioned random point we have generated
        m = 52
        cz,cy,cx =[zz,r,c]
        #grap the volume
        image=scan_1.image_normalized #zxy
        cube_img,corner0 = get_cube_from_img(image, cx, cy, cz, m)
        cube_label,corner1 = get_cube_from_img(scan_1.Z2, cx, cy, cz, m)
        if np.sum(cube_label)==0:
            #save file
            filename=interm_dir3+'data_N_'+str(scan_id)+"_"+str(cx)+"_"+str(cy)+"_"+str(cz)+".pkl"
            with open(filename, 'wb') as f:  # Python 3: open(..., 'wb')
                pickle.dump([cube_img,cube_label.astype(np.bool)], f)
        else:
            k=k-1

#x=[i for i,j in enumerate(scan_id_train) if j==scan_id]
#xx=x[0]

if True:
    temp=!ls {interm_dir3} -irlat #>> myfiles2.csv
    #keep string that satisfy a condition
    temp1=[t for t in temp if "data" in t]
    temp1[0:5]

    temp2=[t.split(" ")[-1] for t in temp1]

    df=pd.DataFrame([t.split(".")[0].split("_")[-4:] for t in temp2],columns=['scan_id','x','y','z'])

    df['label']=0
    df['filename']=temp2
    df.to_csv(tmp_path+ds_type+"df_neg_scanid_centroid.csv")
else:
    df=pd.read_csv(tmp_path+ds_type+"_df_neg_scanid_centroid.csv",index_col=0)
print(df.shape)
df.head(20)

m = 52
for scan_id in scan_id_train:
    #grap the volume
    scan_1 = ctscan(scan_id)
    image=scan_1.image_normalized #zxy
    for c2 in scan_1.centroids2:
        cx,cy,cz =c2
        cube_img,corner0 = get_cube_from_img(image, cx, cy, cz, m)
        cube_label,corner1 = get_cube_from_img(scan_1.Z2, cx, cy, cz, m)
        filename=interm_dir2+'data_P_'+str(scan_id)+"_"+str(cx)+"_"+str(cy)+"_"+str(cz)+".pkl"
        with open(filename, 'wb') as f:  # Python 3: open(..., 'wb')
            pickle.dump([cube_img,cube_label.astype(np.bool)], f)

if True:
    temp=!ls {interm_dir2} -irlat #>> myfiles2.csv
    #keep string that satisfy a condition
    temp1=[t for t in temp if "data" in t]
    temp1[0:5]

    temp2=[t.split(" ")[-1] for t in temp1]

    df=pd.DataFrame([t.split(".")[0].split("_")[-4:] for t in temp2],columns=['scan_id','x','y','z'])
    df['label']=1
    df['filename']=temp2
    #let us compute some analytics
    Area=[]
    for file in df.filename:
        filename=interm_dir2+'/'+file
        with open(filename, 'rb') as f:  # Python 3: open(..., 'wb')
            X,Y = pickle.load(f)
        Area.append(np.sum(Y)/52./52./52.*100)
    df["Area_percentage"] = Area
    
    df.to_csv(tmp_path+ds_type+"df_pos_scanid_centroid.csv")
else:
    df=pd.read_csv(tmp_path+ds_type+"df_pos_scanid_centroid.csv",index_col=0)
print(df.shape)
df.head(20)

dfp=pd.read_csv(tmp_path+ds_type+"df_pos_scanid_centroid.csv",index_col=0)
del dfp["Area_percentage"]
dfp.head()

dfn=pd.read_csv(tmp_path+ds_type+"df_neg_scanid_centroid.csv",index_col=0)
dfn.head()

df2= dfn.sample(frac=.08,random_state=313).reset_index(drop=True)
print(dfn.shape)
print(df2.shape)
df2.head()

df=dfp.append(df2)

df= df.sample(frac=1,random_state=313).reset_index(drop=True)
print(df.shape)
df.head()

df.to_csv(processed_path+fname_df)

now = datetime.datetime.now()
today = now.strftime('%Y-%m-%d')

today = today.replace('-','_')

#write to .rec file
if True:    
    fname=processed_path+ds_type+today+'.rec'
    idx=processed_path+ds_type+today+'.idx'

    record = mx.recordio.MXIndexedRecordIO(idx, fname, 'w')

    counter=0
    sum_img=np.zeros((32,32,32))


    for index, row in df.iterrows():
        if row['label']==1:
            path=interm_dir2
        else:
            path=interm_dir3

        filename=path+'/'+row.filename

        with open(filename, 'rb') as f:  # Python 3: open(..., 'wb')
            x,t=pickle.load(f)
            x,t = random_crop(x,t)



        sum_img+=x

        t=1*t.flatten()#(255*t.flatten()).astype(np.uint8)
        x=x.flatten()#(255*x.flatten()).astype(np.uint8)
        header = mx.recordio.IRHeader(flag=0, label=x, id=int(counter), id2=0)
        s = mx.recordio.pack_img(header, t, quality=100,img_fmt='.png')
        record.write_idx(counter,s)
        counter +=1
    record.close()